# Data Preprocessing

In [21]:
import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch import matmul
from torch.nn.functional import softmax

import tensorflow as tf
from tensorflow import keras

In [22]:
dataframe1 = pd.read_csv('CompleteDataSet.csv', dtype="str")

dataset = dataframe1.values.tolist()

new_dataset = np.array(dataset)

In [23]:
X = []
y = []

for i in range(1,4):
    # Trial
    for j in range(1,12):
        # Activity
        for k in range(1,18):
            # Subject
            z = []
            # Each Video
            for n in range(1,len(new_dataset)-1):
                if (float(new_dataset[n][45]) == i) & (float(new_dataset[n][44]) == j) & (float(new_dataset[n][43]) == k):
                    a = new_dataset[n,15:18]
                    a = a.astype(np.float32)
                    b = new_dataset[n,29:32]
                    b = b.astype(np.float32)
                    c = np.concatenate((a,b), axis = 0)
                    z.append(c)
            X.append(z)
            y.append(j)

In [24]:
X.pop(551)
X.pop(364)
y.pop(551)
y.pop(364)


11

In [25]:
counter = 0
max = 0
min = 1000
for i in range(len(X)):
    counter += len(X[i])
    if len(X[i]) == 0:
        print(i)
    if len(X[i]) > max:
        max = len(X[i])
    if len(X[i]) < min:
        min = len(X[i])

print(f"Avg: {counter/len(X)}\nMin: {min}\nMax: {max}")

Avg: 527.1502683363149
Min: 140
Max: 1145


In [26]:
def to_np_array(X):
    for i in range(len(X)):
        X[i] = np.array(X[i])
        for j in range(len(X[i])):
            X[i][j] = np.array(X[i][j])
    X = np.array(X)
    return X

In [27]:
X_train = []
X_test = []
y_train = []
y_test = []

for i in range(1,12):
    counter = 0
    for j in range(len(y)):
        if (i == y[j]) & (counter < 7):
            X_test.append(X[j])
            y_test.append(y[j])
            counter += 1
        elif (i == y[j]) & (counter >= 7):
            X_train.append(X[j])
            y_train.append(y[j])
            counter += 1

X_train = to_np_array(X_train)
X_test = to_np_array(X_test)
y_train = np.array(y_train, dtype=np.uint8)
y_test = np.array(y_test, dtype=np.uint8)

C:\Users\user\AppData\Local\Temp\ipykernel_18824\3925073110.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(X)


In [28]:
print(len(X_train))
print(len(X_test))

482
77


In [29]:
# 1:Fall    0:NotFall
def to_binary(y):
  for i in range(len(y)):
    if (y[i] >= 6) & (y[i] <= 11):
      y[i] = 0
    elif (y[i] >= 1) & (y[i] <= 5):
      y[i] = 1
    else:
      print(i)
  return y

y_train = to_binary(y_train)
y_test = to_binary(y_test)

In [30]:
print(type(X_train))
print(type(X_train[0]))
print(type(X_train[0][0]))
print(type(X_train[0][0][0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float32'>


# PyTorch

## DataLoader

In [31]:
X_train_tensor = torch.zeros((len(X_train),max, 6))
X_test_tensor = torch.zeros((len(X_test),max, 6))
y_train_tensor = torch.Tensor(y_train)
y_test_tensor = torch.Tensor(y_test)

for i in range(len(X_train)):
    for j in range(len(X_train[i])):
        for k in range(len(X_train[i][j])):
            X_train_tensor[i][j][k] = float(X_train[i][j][k])
for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        for k in range(len(X_test[i][j])):
            X_test_tensor[i][j][k] = float(X_test[i][j][k])

print(X_train_tensor.shape)
print(X_test_tensor.shape)

torch.Size([482, 1145, 6])
torch.Size([77, 1145, 6])


In [32]:
train_dataset = []
for i in range(len(X_train_tensor)):
    a = (X_train_tensor[i], y_train_tensor[i])
    train_dataset.append(a)
test_dataset = []
for i in range(len(X_test_tensor)):
    a = (X_test_tensor[i], y_test_tensor[i])
    test_dataset.append(a)

In [33]:
# Device configuration
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 19

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## RNN

In [34]:
# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(p=0.4)

        # or:
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        # self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, num_classes)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        out, _ = self.gru(x, h0)  
        # or:
        # out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.relu(out)
        # out = self.fc2(out)
        # out = self.relu(out)
        # out = self.fc2(out)
        # out = self.relu(out)
        out = self.fc1(out)
        out = self.softmax(out)
        # out: (n, 10)
        return out

# num_classes = 2
# input_size = 6
# hidden_size = 12
# num_layers = 2
# rnn = RNN(input_size, hidden_size, num_layers, num_classes)

## Attention

In [ ]:
class SelfAttention(nn.Module):
  
  def __init__(self, emb_dim, model_dim):
    super(SelfAttention, self).__init__()
    self.emb_dim = emb_dim
    self.model_dim = model_dim
    
    self.to_query = nn.Linear(emb_dim, model_dim, bias=True)
    # self.to_query.weight = nn.Parameter(w_query.t())
    
    self.to_key = nn.Linear(emb_dim, model_dim, bias=True)
    # self.to_key.weight = nn.Parameter(w_key.t())
    
    self.to_value = nn.Linear(emb_dim, model_dim, bias=True)
    # self.to_value.weight = nn.Parameter(w_value.t())

  
  def forward(self, inputs):
    q = self.to_query(inputs)
    k = self.to_key(inputs)
    v = self.to_value(inputs)

    attn_score = matmul(q, k.transpose(-1, -2))
    softmax_attn_score = softmax(attn_score, dim=-1)
    output = matmul(softmax_attn_score, v)

    return output

In [41]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_model, heads=2, p=0.4, num_classes=2):
    super(MultiHeadAttention, self).__init__()
    self.heads = heads
    self.d_model = d_model
    heads_dim = int(d_model / heads)
    self.heads_dim = heads_dim
    self.p = p
    self.num_classes = num_classes

    self.to_query = nn.Linear(d_model, heads * heads_dim, bias=True)
    self.to_key = nn.Linear(d_model, heads * heads_dim, bias=True)
    self.to_value = nn.Linear(d_model, heads * heads_dim, bias=True)

    self.unify_heads = nn.Linear(heads * heads_dim, d_model, bias=True)
    self.fc_final = nn.Linear(d_model, 2)
    self.softmax = nn.Softmax(dim=1)

    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=p)
    self.norm = nn.LayerNorm(self.d_model)
  
  def forward(self, inputs, mask=None, kv=None):
    batch_size, sequence_length, emb_dim = inputs.shape
    
    if kv is not None:
      kv = kv
    else:
      kv = inputs
    kv_batch_szie, kv_sequence_lentgh, _ = kv.size()

    # Head
    inputs = self.norm(inputs)

    q = self.to_query(inputs).view(batch_size, sequence_length, self.heads, self.heads_dim).transpose(1, 2)
    k = self.to_key(kv).view(kv_batch_szie, kv_sequence_lentgh, self.heads, self.heads_dim).transpose(1, 2)
    v = self.to_value(kv).view(kv_batch_szie, kv_sequence_lentgh, self.heads, self.heads_dim).transpose(1, 2)

    attn_score = matmul(q, k.transpose(-1, -2))
    
    # Scale After Dot-Product
    attn_score = attn_score / self.heads_dim ** (1/float(2))
    
    # Mask
    if mask is not None:
      attn_score = attn_score.masked_fill(mask==1, value=-1e9)
    
    # Softmax
    softmax_attn_score = softmax(attn_score, dim=-1)
    
    output = matmul(softmax_attn_score, v)
    output = output.transpose(1, 2).contiguous().view(batch_size, sequence_length, self.heads * self.heads_dim)
    output_final = self.unify_heads(output)
    
    # output_final = self.relu(output_final)
    # output_final = self.dropout(output_final)
    # predictions = self.fc_final(output_final)
    # predictions = self.softmax(predictions)
    
    num_classes = 2
    input_size = 6
    hidden_size = 12
    num_layers = 2
    rnn = RNN(input_size, hidden_size, num_layers, num_classes)
    output_final = rnn(output_final)
    
    return output_final

In [10]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_model, heads=2, p=0.4, num_classes=2):
    super(MultiHeadAttention, self).__init__()
    # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    # -> x needs to be: (batch_size, seq, input_size)
    self.softmax = nn.Softmax(dim=1)
    self.dropout = nn.Dropout(p=0.4)

    self.input_size = 6
    self.hidden_size = 12
    self.num_layers = 2
    self.num_classes = num_classes
    # or:
    # self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
    self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
    self.fc1 = nn.Linear(self.hidden_size, self.num_classes)
    self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)

    self.heads = heads
    self.d_model = d_model
    heads_dim = int(d_model / heads)
    self.heads_dim = heads_dim
    self.p = p

    self.to_query = nn.Linear(d_model, heads * heads_dim, bias=True)
    self.to_key = nn.Linear(d_model, heads * heads_dim, bias=True)
    self.to_value = nn.Linear(d_model, heads * heads_dim, bias=True)

    self.unify_heads = nn.Linear(heads * heads_dim, d_model, bias=True)
    self.fc_final = nn.Linear(d_model, 2)
    self.softmax = nn.Softmax(dim=1)

    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=p)
    self.norm = nn.LayerNorm(self.d_model)
  
  def forward(self, inputs, mask=None, kv=None):
    batch_size, sequence_length, emb_dim = inputs.shape
    
    if kv is not None:
      kv = kv
    else:
      kv = inputs
    kv_batch_szie, kv_sequence_lentgh, _ = kv.size()

    # Head
    inputs = self.norm(inputs)

    q = self.to_query(inputs).view(batch_size, sequence_length, self.heads, self.heads_dim).transpose(1, 2)
    k = self.to_key(kv).view(kv_batch_szie, kv_sequence_lentgh, self.heads, self.heads_dim).transpose(1, 2)
    v = self.to_value(kv).view(kv_batch_szie, kv_sequence_lentgh, self.heads, self.heads_dim).transpose(1, 2)

    attn_score = matmul(q, k.transpose(-1, -2))
    
    # Scale After Dot-Product
    attn_score = attn_score / self.heads_dim ** (1/float(2))
    
    # Mask
    if mask is not None:
      attn_score = attn_score.masked_fill(mask==1, value=-1e9)
    
    # Softmax
    softmax_attn_score = softmax(attn_score, dim=-1)
    
    output = matmul(softmax_attn_score, v)
    output = output.transpose(1, 2).contiguous().view(batch_size, sequence_length, self.heads * self.heads_dim)
    output_final = self.unify_heads(output)
    
    # output_final = self.relu(output_final)
    # output_final = self.dropout(output_final)
    # predictions = self.fc_final(output_final)
    # predictions = self.softmax(predictions)
    
    x = output_final
    # Set initial hidden states (and cell states for LSTM)
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
    
    # x: (n, 28, 28), h0: (2, n, 128)
    
    # Forward propagate RNN
    # out, _ = self.gru(x, h0)  
    # or:
    out, _ = self.lstm(x, (h0,c0))  
    
    # out: tensor of shape (batch_size, seq_length, hidden_size)
    # out: (n, 28, 128)
    
    # Decode the hidden state of the last time step
    out = out[:, -1, :]
    # out: (n, 128)
    out = self.dropout(out)
    out = self.fc2(out)
    out = self.relu(out)
    # out = self.fc2(out)
    # out = self.relu(out)
    # out = self.fc2(out)
    # out = self.relu(out)
    out = self.fc1(out)
    out = self.softmax(out)
    
    return output_final

In [17]:
torch.save(model, "GRU.pt")

RuntimeError: File GRU.pt cannot be opened.

In [13]:
model = MultiHeadAttention(d_model=6, heads=1)
# test_sample = X_test_tensor[:2]
test_sample = torch.randn((2, 1145, 6))

pad = 0
# trg = torch.tensor([[1, 2, 0, 0, 0], [1, 2, 3, 0, 0]])
bs, seq_len, emb_dim = test_sample.shape
mask_shape = (1, seq_len, seq_len, emb_dim)
ones_tensor = torch.ones(mask_shape)
triu_tesnor = torch.triu(ones_tensor, diagonal=1).type(torch.int16)
trg_mask = (test_sample == pad).type(torch.int16).unsqueeze(-2)
mask = triu_tesnor | trg_mask
mask = mask.unsqueeze(1)

# output = model(test_sample, mask)
output = model(test_sample)
print(output[:,:,0].shape)
# print(y_test[:2].long())

torch.Size([2, 1145])


In [ ]:
multi_head_attention = MultiHeadAttention(d_model=6, heads=1)

sample_dataset = test_dataset[:4]

sample_loader = torch.utils.data.DataLoader(dataset=sample_dataset, 
                                            batch_size=4, 
                                            shuffle=False)

for sample, label in sample_loader:
    # print(sample.shape)
    # print(label.shape)
    criterion = nn.CrossEntropyLoss()
    outputs = multi_head_attention(sample)
    outputs = rnn(outputs)
    print(outputs)
    # print(outputs[:,:,0].shape)
    _, predicted = torch.max(outputs.data, 1)
    print(predicted)
    # print(predicted[:,1])
    # print(label.long())
    # print(label.long().shape)
    # loss = criterion(outputs[:,:,0], label.long())

tensor([[0.3748, 0.6252],
        [0.3808, 0.6192],
        [0.3658, 0.6342],
        [0.3795, 0.6205]], grad_fn=<SoftmaxBackward0>)
tensor([1, 1, 1, 1])


## Transformers

### Encoder

In [ ]:
class Encoder(nn.Module):
  def __init__(self, d_model, heads, ff_size=2096, dropout=0.1):
    super(Encoder, self).__init__()

    self.attention = MultiHeadAttention(d_model, heads)
    self.attn_norm = nn.LayerNorm(d_model)

    self.ff = nn.Sequential(
      nn.Linear(d_model, ff_size),
      nn.ReLU(),
      nn.Dropout(dropout),
      nn.Linear(ff_size, d_model)
    )
    
    self.final_norm = nn.LayerNorm(d_model)
    self.do = nn.Dropout(dropout)
  

  def forward(self, src_embeddings, src_mask=None):
    tensor = tensor + self.do(self.attention(self.attn_norm(src_embeddings), src_mask))

    tensor = tensor + self.do(self.ff(self.final_norm(tensor)))

    return tensor

In [ ]:
class TransformerEncoder(nn.Module):
  def __init__(self, d_model, heads, num_layers=2, ff_size=2096, dropout=0.1):
    super(TransformerEncoder, self).__init__()

    self.layers = nn.ModuleList()
    for i in range(num_layers):
      self.layers.append(Encoder(d_model, heads, ff_size, dropout))
    self.norm = nn.LayerNorm(d_model)

  
  def forward(self, src_embeddings, src_mask=None):
    hidden_states = []
    
    for layer in self.layers:
      src_embeddings = layer(src_embeddings, src_mask)
      hidden_states.append(src_embeddings)
    
    return hidden_states, self.norm(hidden_states[-1])

### Decoder

In [ ]:
class Decoder(nn.Module):
  def __init__(self, d_model, heads, ff_size=2096, dropout=0.1):
    super(Encoder, self).__init__()

    self.attention = MultiHeadAttention(d_model, heads)
    self.attn_norm = nn.LayerNorm(d_model)

    self.encoder_decoder_attn = MultiHeadAttention(d_model, heads)
    self.encoder_decoder_norm = nn.LayerNorm(d_model)

    self.ff = nn.Sequential(
      nn.Linear(d_model, ff_size),
      nn.ReLU(),
      nn.Dropout(dropout),
      nn.Linear(ff_size, d_model)
    )
    
    self.final_norm = nn.LayerNorm(d_model)
    self.do = nn.Dropout(dropout)
  

  def forward(self, src_embeddings, memory, src_mask=None, trg_mask=None):
    tensor = tensor + self.do(self.attention(self.attn_norm(src_embeddings), src_mask))
    tensor = tensor + self.do(self.encoder_decoder_attn(self.encoder_decoder_norm(src_embeddings), kv=memory, mask=src_mask))
 
    tensor = tensor + self.do(self.ff(self.final_norm(tensor)))

    return tensor

In [ ]:
class TransformerDecoder(nn.Module):
  def __init__(self, d_model, heads, num_layers=2, ff_size=2096, dropout=0.1):
    super(TransformerEncoder, self).__init__()

    self.layers = nn.ModuleList()
    for i in range(num_layers):
      self.layers.append(Decoder(d_model, heads, ff_size, dropout))
    self.norm = nn.LayerNorm(d_model)

  
  def forward(self, src_embeddings, memory, src_mask=None, trg_mask=None):
    hidden_states = []
    
    for layer in self.layers:
      src_embeddings = layer(src_embeddings, memory, src_mask, trg_mask)
      hidden_states.append(src_embeddings)
    
    return hidden_states, self.norm(hidden_states[-1])

In [ ]:
sequence_length, d_model = test_sample
model_encoder = TransformerEncoder(d_model=d_model, heads=1, num_layers=3)
hidden_encoder, output_encoder = model_encoder(test_sample)

model_decoder = TransformerDecoder(d_model=d_model, heads=1, num_layers=3)
hidden_decoder, output_decoder = model_decoder(test_sample, output_encoder)

ValueError: only one element tensors can be converted to Python scalars

## Refrence

In [ ]:
class MultiheadedAttention(nn.Module):
    def __init__(self, d_model, heads=2):
        super(MultiheadedAttention, self).__init__()

        self.heads = heads

        # The model dimension split into n-heads
        self.heads_dim = int(d_model / heads)

        # In final implementation, we must use bias=True
        self.to_query = nn.Linear(d_model, heads * self.heads_dim, bias=False)
        # self.to_query.weight = nn.Parameter(w_query.t())  # This should be commented in final implementation

        # In final implementation, we must use bias=True
        self.to_key = nn.Linear(d_model, heads * self.heads_dim, bias=False)
        # self.to_key.weight = nn.Parameter(w_key.t())  # This should be commented in final implementation

        # In final implementation, we must use bias=True
        self.to_value = nn.Linear(d_model, heads * self.heads_dim, bias=False)
        # self.to_value.weight = nn.Parameter(w_value.t())  # This should be commented in final implementation

        # In final implementation, we must use bias=True
        self.unify_heads = nn.Linear(heads * self.heads_dim, d_model, bias=False)
        # self.unify_heads.weight = nn.Parameter(w_unify_heads.t())  # This should be commented in final implementation

    def forward(self, inputs, mask=None, kv=None):
        # Create Q, K, and V using input vectors
        bs, seq, emb_dim = inputs.shape

        if kv is not None:
            kv = kv
        else:
            kv = inputs

        kv_bs, kv_seq_len, _ = kv.size()

        # Transpose: bs x seq-length x num-heads x heads_dim -> bs x num-heads x seq-length x heads_dim
        q = self.to_query(inputs).view(bs, seq, self.heads, self.heads_dim).transpose(1, 2)
        k = self.to_key(kv).view(kv_bs, kv_seq_len, self.heads, self.heads_dim).transpose(1, 2)
        v = self.to_value(kv).view(kv_bs, kv_seq_len, self.heads, self.heads_dim).transpose(1, 2)

        # Scale before Dot-product: q/root_forth(head_dim) and k/root_forth(head_dim)
        q = q / (self.heads_dim ** 1 / float(4))
        k = k / (self.heads_dim ** 1 / float(4))

        # Compute Attention scores
        attn_scores = matmul(q, k.transpose(-1, -2))
        # Scale after Dot-product : attn_scores/root_square(head_dim)
        # attn_scores = attn_scores/(self.heads_dim ** 1/float(2))

        # Apply masking
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 1, value=-1e9)

        # Convert attention scores into probability distributions
        softmax_attn_scores = softmax(attn_scores, dim=-1)

        # Compute Weighted Values
        output = matmul(softmax_attn_scores, v)

        # Reshape the weighted values
        # Transpose: bs x seq-length x num-heads x heads_dim -> bs x seq-length x num-heads x heads_dim)
        output = output.transpose(1, 2).contiguous().view(bs, seq, self.heads * self.heads_dim)
        output_final = self.unify_heads(output)
        return output_final

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, ff_size=2096, dropout=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = MultiheadedAttention(d_model=d_model, heads=num_heads)
        self.attn_norm = nn.LayerNorm(d_model)

        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(ff_size, d_model)
        )

        self.final_norm = nn.LayerNorm(d_model)

        self.do = nn.Dropout(dropout)

    def forward(self, embeddings, mask=None):
        # Attn with Pre-Normalization
        embeddings = embeddings + self.do(self.self_attn(self.attn_norm(embeddings), mask=mask))
        # FeedForward with Pre-Normalization
        embeddings = embeddings + self.do(self.ff(self.final_norm(embeddings)))

        return embeddings


class TransformerEncoder(nn.Module):
    def __init__(self, d_model, num_heads=2, num_layers=2):
        super(TransformerEncoder, self).__init__()

        self.enc_layers = nn.ModuleList()
        for i in range(num_layers):
            self.enc_layers.append(TransformerEncoderLayer(d_model, num_heads=num_heads))

        self.norm = nn.LayerNorm(d_model)

    def forward(self, embeddings, mask=None):

        hidden_states = []
        for layer in self.enc_layers:
            embeddings = layer(embeddings, mask)
            hidden_states.append(embeddings)

        return hidden_states, self.norm(hidden_states[-1])


# if __name__ == '__main__':
#     dim_model = 512
#     src_embeddings = torch.rand(2, 32, dim_model)
#     model_encoder = TransformerEncoder(d_model=dim_model, num_heads=8, num_layers=6)
#     hidden, output = model_encoder(src_embeddings)
#     print(len(hidden))
#     print(output.shape)

In [ ]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, ff_size=2096, dropout=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.self_attn = MultiheadedAttention(d_model=d_model, heads=num_heads)
        self.attn_norm = nn.LayerNorm(d_model)

        self.encoder_decoder_attn = MultiheadedAttention(d_model=d_model, heads=num_heads)

        self.encoder_decoder_norm = nn.LayerNorm(d_model)

        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(ff_size, d_model)
        )

        self.final_norm = nn.LayerNorm(d_model)

        self.do = nn.Dropout(dropout)

    def forward(self, embeddings, memory, src_mask=None, trg_mask=None):
        # Attn with Pre-Normalization
        embeddings = embeddings + self.do(self.self_attn(self.attn_norm(embeddings), mask=trg_mask))

        # Cross Attn with Pre-Normalization
        embeddings = embeddings + self.do(self.encoder_decoder_attn(self.encoder_decoder_norm(embeddings), kv=memory, mask=src_mask))

        # FeedForward with Pre-Normalization
        embeddings = embeddings + self.do(self.ff(self.final_norm(embeddings)))

        return embeddings


class TransformerDecoder(nn.Module):
    def __init__(self, d_model, num_heads=2, num_layers=2):
        super(TransformerDecoder, self).__init__()

        self.enc_layers = nn.ModuleList()
        for i in range(num_layers):
            self.enc_layers.append(TransformerDecoderLayer(d_model, num_heads=num_heads))

        self.norm = nn.LayerNorm(d_model)

    def forward(self, embeddings, memory, src_mask=None, trg_mask=None):

        hidden_states = []
        for layer in self.enc_layers:
            embeddings = layer(embeddings, memory, src_mask, trg_mask)
            hidden_states.append(embeddings)

        return hidden_states, self.norm(hidden_states[-1])


if __name__ == '__main__':
    # dim_model = 512
    # src_embeddings = torch.rand(2, 32, dim_model)
    src_embeddings = X_test_tensor[:10]
    batch_size, sequence_length, dim_model = src_embeddings.shape
    model_encoder = TransformerEncoder(d_model=dim_model, num_heads=1, num_layers=2)
    model_decoder = TransformerDecoder(d_model=dim_model, num_heads=1, num_layers=2)
    hidden, output = model_encoder(src_embeddings)
    print(len(hidden))
    print(output.shape)

    trg_embeddings = torch.rand(batch_size, sequence_length, dim_model)
    hidden, output = model_decoder(trg_embeddings, output)
    print(len(hidden))
    print(output.shape)

## Training

In [37]:
learning_rate = 0.0001
num_epochs = 5


num_classes = 2
input_size = 6
hidden_size = 12
num_layers = 2
rnn = RNN(input_size, hidden_size, num_layers, num_classes)
# model = MultiHeadAttention(6, 1)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        images = images
        labels = labels
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels.long())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


RuntimeError: Expected target size [19, 6], got [19]

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        # images = images.to(device)
        # labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the {len(test_dataset)} test images: {acc} %')

NameError: name 'test_loader' is not defined

In [ ]:
num_epochs = 10
learning_rate = 0.001
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for j in range(1,6):
    max = 0
    for i in range(10):
        model = MultiHeadAttention(6, j)

        # Train the model
        n_total_steps = len(train_loader)
        for epoch in range(num_epochs):
            for i, (images, labels) in enumerate(train_loader):  
                # origin shape: [N, 1, 28, 28]
                # resized: [N, 28, 28]
                images = images
                labels = labels
                
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels.long())
                
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

        # Test the model
        # In test phase, we don't need to compute gradients (for memory efficiency)
        with torch.no_grad():
            n_correct = 0
            n_samples = 0
            for images, labels in test_loader:
                # images = images.to(device)
                # labels = labels.to(device)
                outputs = model(images)
                # max returns (value ,index)
                _, predicted = torch.max(outputs.data, 1)
                n_samples += labels.size(0)
                n_correct += (predicted == labels).sum().item()

            acc = 100.0 * n_correct / n_samples
            # print(f'Accuracy of the network on the {len(test_dataset)} test images: {acc} %')
        
        if acc > max:
            # torch.save(f"attention{j}.pt", model)
            max = acc
    print(f"{j}HeadAttention: {max}%")

1HeadAttention: 94.8051948051948%
2HeadAttention: 94.8051948051948%
3HeadAttention: 77.92207792207792%


In [7]:
model = MultiHeadAttention(6, 1)
optimizer = torch.optim.Adam(model.parameters())

# Additional information
EPOCH = 5
PATH = "model.pt"
LOSS = 0.4

torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
            }, PATH)

In [8]:
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()

MultiHeadAttention(
  (to_query): Linear(in_features=6, out_features=6, bias=True)
  (to_key): Linear(in_features=6, out_features=6, bias=True)
  (to_value): Linear(in_features=6, out_features=6, bias=True)
  (unify_heads): Linear(in_features=6, out_features=6, bias=True)
  (fc_final): Linear(in_features=6, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (norm): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
)

In [9]:
model = MultiHeadAttention(6, 1)
PATH = "1HeadAttention.pt"
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
# model = MultiHeadAttention(6, 1)

example = torch.Tensor(batch_size, max, input_size)

traced_script_module = torch.jit.trace(model, example)

output = traced_script_module(torch.ones(batch_size, max, input_size))

sm = torch.jit.script(model)

traced_script_module.save("Attention.pt")